In [6]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq
import csv
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [2]:
# Groq API and Models 
Groq_Token = "API_Key_here"

groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

In [4]:
examples = f"""
accx = -0.1994058, accy = -0.03393813, accz = 0.9722126 : LAYING
accx = 0.9922325, accy = 0.0677996, accz = 0.2404148 : SITTING
accx = 0.9571659, accy = -0.1973788, accz = -0.3444446 : STANDING
accx = 1.073806, accy = -0.005115331, accz = 0.1358182 : WALKING
accx = 0.8900631, accy = 0.0321876, accz = -0.1827345 : WALKING_DOWNSTAIRS
accx = 1.078195, accy = -0.2588233, accz = -0.1880357 : WALKING_UPSTAIRS
"""

#using Arul's Data to test
test_samples = ["accx = 0.0129, accy = -0.0466, accz = 0.0172",
                "accx = 0.1186, accy = 0.0242, accz = -0.0934",
                "accx = 0.02, accy = 0.013, accz = 0.0405",
                "accx = -3.159, accy = 0.0941, accz = 1.6808",
                "accx = 2.46, accy = -0.8987, accz = -2.1262",
                "accx = 3.9627, accy = -3.1137, accz = -3.9568"]
#LAYING, SITTING, STANDING, WALKING, WALKING_DOWNSTAIRS, WALKING_UPSTAIRS

query = f"""
* You are a activity recognition model. 
* Your task is to analyze the given data in form of accelerations in x, y and z directions and classify the activity as 'LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS' or 'WALKING_UPSTAIRS'. 
* Provide the activity label and, if necessary, a brief explanation of your reasoning.

A few examples are as follows:
{examples}

Analyze the following data and predict the activities:
1. {test_samples[0]}
2. {test_samples[1]}
3. {test_samples[2]}
4. {test_samples[3]}
5. {test_samples[4]}
6. {test_samples[5]}
""" 

#we use the same models we used for Task 3 to maintain consistency
model_name = "llama3.1-8b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("Answer from llama3.1-8b: ")
print(answer.content)

model_name = "llama3.1-70b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("\n Answer from llama3.1-70b: ")
print(answer.content)

Answer from llama3.1-8b: 
Based on the given data, I will analyze each sample and predict the activity.

1. accx = 0.0129, accy = -0.0466, accz = 0.0172
This sample has very low acceleration values in all directions, which is typical for a person who is LAYING. The accelerations are close to zero, indicating minimal movement.

**Predicted Activity:** LAYING

2. accx = 0.1186, accy = 0.0242, accz = -0.0934
This sample has moderate acceleration values in the x and y directions, but a relatively low value in the z direction. This suggests that the person is moving, but not in a way that would be typical for walking or running. The accelerations are not high enough to indicate STANDING or WALKING.

**Predicted Activity:** SITTING

3. accx = 0.02, accy = 0.013, accz = 0.0405
This sample has very low acceleration values in all directions, which is similar to the first sample. However, the accelerations are slightly higher than the first sample, but still not high enough to indicate movement.

In [9]:
y_test = pd.DataFrame(['LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS'])

#llama3.1-8b
y_pred = pd.DataFrame(['LAYING', 'SITTING', 'LAYING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'WALKING_DOWNSTAIRS'])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'llama3.1-8b :')
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

#llama3.1-70b
y_pred = pd.DataFrame(['LAYING', 'SITTING', 'LAYING', 'WALKING_DOWNSTAIRS', 'WALKING_DOWNSTAIRS', 'WALKING_DOWNSTAIRS'])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'llama3.1-70b :')
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

llama3.1-8b :
Accuracy: 0.50
Precision: 0.33
Recall: 0.50
llama3.1-70b :
Accuracy: 0.50
Precision: 0.31
Recall: 0.50


In [5]:
examples = f"""
accx = -0.1994058, accy = -0.03393813, accz = 0.9722126 : LAYING
accx = -0.02925219, accy = 0.7027735, accz = 0.7284641 : LAYING
accx = 0.9922325, accy = 0.0677996, accz = 0.2404148 : SITTING
accx = 1.02311, accy = -0.02337259, accz = -0.06775652 : SITTING
accx = 0.9571659, accy = -0.1973788, accz = -0.3444446 : STANDING
accx = 1.012637, accy = -0.1900052, accz = 0.05822079 : STANDING
accx = 1.073806, accy = -0.005115331, accz = 0.1358182 : WALKING
accx = 1.415695, accy = 0.05032975, accz = -0.06400823 : WALKING
accx = 0.8900631, accy = 0.0321876, accz = -0.1827345 : WALKING_DOWNSTAIRS
accx = 1.304819, accy = -0.2678744, accz = 0.3007964 : WALKING_DOWNSTAIRS
accx = 1.078195, accy = -0.2588233, accz = -0.1880357 : WALKING_UPSTAIRS
accx = 1.479692, accy = -0.6064999, accz = 0.06833482 : WALKING_UPSTAIRS
"""

#using Arul's Data to test
test_samples = ["accx = 0.0129, accy = -0.0466, accz = 0.0172",
                "accx = 0.1186, accy = 0.0242, accz = -0.0934",
                "accx = 0.02, accy = 0.013, accz = 0.0405",
                "accx = -3.159, accy = 0.0941, accz = 1.6808",
                "accx = 2.46, accy = -0.8987, accz = -2.1262",
                "accx = 3.9627, accy = -3.1137, accz = -3.9568"]
#LAYING, SITTING, STANDING, WALKING, WALKING_DOWNSTAIRS, WALKING_UPSTAIRS

query = f"""
* You are a activity recognition model. 
* Your task is to analyze the given data in form of accelerations in x, y and z directions and classify the activity as 'LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS' or 'WALKING_UPSTAIRS'. 
* Provide the activity label and, if necessary, a brief explanation of your reasoning.

A few examples are as follows:
{examples}

Analyze the following data and predict the activities:
1. {test_samples[0]}
2. {test_samples[1]}
3. {test_samples[2]}
4. {test_samples[3]}
5. {test_samples[4]}
6. {test_samples[5]}
""" 

#we use the same models we used for Task 3 to maintain consistency
model_name = "llama3.1-8b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("Answer from llama3.1-8b: ")
print(answer.content)

model_name = "llama3.1-70b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print("\n Answer from llama3.1-70b: ")
print(answer.content)

Answer from llama3.1-8b: 
Based on the given data, I will analyze each sample and predict the activity.

1. accx = 0.0129, accy = -0.0466, accz = 0.0172
This sample has a low acceleration in the x-direction and a small negative acceleration in the y-direction. The z-direction acceleration is also low. These characteristics are similar to those of 'LAYING'. Therefore, I predict the activity as 'LAYING'.

2. accx = 0.1186, accy = 0.0242, accz = -0.0934
This sample has a moderate acceleration in the x-direction and a small positive acceleration in the y-direction. The z-direction acceleration is slightly negative. These characteristics are similar to those of 'SITTING'. Therefore, I predict the activity as 'SITTING'.

3. accx = 0.02, accy = 0.013, accz = 0.0405
This sample has a low acceleration in the x-direction and a small positive acceleration in the y-direction. The z-direction acceleration is also low. These characteristics are similar to those of 'LAYING'. Therefore, I predict the 

In [10]:
y_test = pd.DataFrame(['LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS'])

#llama3.1-8b
y_pred = pd.DataFrame(['LAYING', 'SITTING', 'LAYING', 'LAYING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS'])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'llama3.1-8b :')
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

#llama3.1-70b
y_pred = pd.DataFrame(['LAYING', 'LAYING', 'LAYING', 'WALKING_DOWNSTAIRS', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS'])

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'llama3.1-70b :')
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

llama3.1-8b :
Accuracy: 0.67
Precision: 0.56
Recall: 0.67
llama3.1-70b :
Accuracy: 0.17
Precision: 0.06
Recall: 0.17
